### Predict near future maximum price

When RSI $\ge$ 70, the stock may attain a local maximum in the near future. 
This may be a good time to sell. However, one needs to decide the selling price. 
This article attempts to use historical data to predict this maximum price.

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from ta.trend import MACD
from ta.momentum import StochasticOscillator

import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
def calculate_rsi_sma(data, period=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def smma(series, period):
    smma = series.copy()
    smma.iloc[:period] = series.iloc[:period].mean()  # seed with SMA
    for i in range(period, len(series)):
        smma.iloc[i] = ((smma.iloc[i - 1] * (period - 1)) + series.iloc[i]) / period
    return smma

def calculate_rsi_smma(data, period=14):
    delta = data.diff()

    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = smma(gain, period)
    avg_loss = smma(loss, period)

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def calculate_rsi_ema(data, period=14):
    # Calculate price change
    delta = data.diff()

    # Separate gains and losses
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    # Apply Exponential Moving Average
    avg_gain = gain.ewm(span=period, adjust=False).mean()
    avg_loss = loss.ewm(span=period, adjust=False).mean()

    # Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss

    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

def prepare_df(ticker, days=0, start_date='2024-09-01', end_date='2025-08-31', average_method='sma'):
    '''
    average_method: 
        'sma' for Simple Moving Average
        'smma' for Smoothing Moving Average
        'ema' for Exponential Moving Average
    '''
    if days > 0:
        df = yf.download(ticker, period=f'{days}d', auto_adjust=True, progress=False)
    else:
        df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
    
    # keep only the "Open", "Close" etc for plotly. also remove the unnecessary [] of df['Close'].values
    df.columns = df.columns.droplevel(1)    

    if average_method == 'sma':
        df['RSI'] = calculate_rsi_sma(df['Close'])
    elif average_method == 'smma':
        df['RSI'] = calculate_rsi_smma(df['Close'])
    elif average_method == 'ema':
        df['RSI'] = calculate_rsi_ema(df['Close'])
    else:
        df['RSI'] = calculate_rsi_sma(df['Close'])

    # add moving averages
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA5'] = df['Close'].rolling(window=5).mean()

    # MACD
    macd = MACD(close=df['Close'],
                      window_slow=26,
                      window_fast=12,
                      window_sign=9)
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Diff'] = macd.macd_diff()

    # stochastic
    stoch = StochasticOscillator(high=df['High'],
                                       close=df['Close'],
                                       low=df['Low'],
                                       window=14,
                                       smooth_window=3)
    df['Stoch'] = stoch.stoch()
    df['Stoch_Signal'] = stoch.stoch_signal()

    df['Volume_log'] = np.log1p(df['Volume'])
    mean_volume_log = df['Volume_log'].mean()
    std_volume_log = df['Volume_log'].std()
    df['Volume_log_Norm'] = (df['Volume_log'] - mean_volume_log) / std_volume_log

    return df, mean_volume_log, std_volume_log

In [3]:
SYMBOLS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']
dfs = {}
mean_volume_logs = {}
std_volume_logs = {}
for symbol in SYMBOLS:
    dfs[symbol], mean_volume_logs[symbol], std_volume_logs[symbol] = prepare_df(symbol, start_date='2022-09-01', end_date='2025-08-31', average_method='sma')

In [4]:
TRADE_DAYS_PER_YEAR = 251
LEFT_MIN = 21       # trade days per month
LEFT_AVERAGE = 21
LEFT_MAX = 5        # arbitrary take
RIGHT_MAX = 5       # arbitrary take

In [5]:
def get_price_average(df, start_index, end_index):
    return df.iloc[start_index:end_index]['Close'].mean()

def get_price_min(df, start_index, end_index):
    return df.iloc[start_index:end_index]['Close'].min()

def get_price_max(df, start_index, end_index):
    return df.iloc[start_index:end_index]['Close'].max()


#### Extract data where RSI $\ge$ 70
Data include
1. Today and yesterday Close Price.
2. Recent minimum, average and maximum prices.
3. Today and last few days RSI.
4. Volume in normalized logarithm.

In [6]:
def extract_RSI70_data(df, printout=False):
    # skip beginning until RSI < 70
    for i_skip, row in df[max(LEFT_MIN, LEFT_AVERAGE, LEFT_MAX):].iterrows():   # skip enough data for calculate min, average, max
        if row['RSI'] < 70:
            break
    
    count = 0
    consecutive_days = 0
    price_div_max_sum = 0
    price_next_div_max_sum = 0
    price_div_min_sum = 0
    price_next_div_min_sum = 0
    price_div_average_sum = 0
    price_next_div_average_sum = 0
    data = []
    inputs = []
    outputs = []
    if printout:
        print('count, date, index, close, close_prev, price_min, price_average, price_max, rsi, rsi_prev, rsi_prev2, rsi_prev3, rsi_prev4')
    for i, row in df[df.index.get_loc(i_skip):].iterrows(): 
        iloc_index = df.index.get_loc(i)
        if row['RSI'] >= 70:
            if consecutive_days == 0:
                count += 1
            consecutive_days += 1
            
            if consecutive_days == 1:
                d = {}
                d['Date'] = i.strftime('%Y-%m-%d')
                d['Index'] = iloc_index
                d['Close'] = row['Close']
                d['Close_Prev'] = df.iloc[iloc_index-1]['Close']
                d['Min'] = get_price_min(df, iloc_index - LEFT_MIN, iloc_index + 1)
                d['Average'] = get_price_average(df, iloc_index - LEFT_AVERAGE, iloc_index + 1)
                d['Max'] = get_price_max(df, iloc_index - LEFT_MAX, iloc_index + RIGHT_MAX + 1)
                d['RSI'] = row['RSI']
                d['RSI_Prev'] = df.iloc[iloc_index-1]['RSI']
                d['RSI_Prev2'] = df.iloc[iloc_index-2]['RSI']
                d['RSI_Prev3'] = df.iloc[iloc_index-3]['RSI']
                d['RSI_Prev4'] = df.iloc[iloc_index-4]['RSI']
                d['Volume_log_Norm'] = row['Volume_log_Norm']
                data.append(d)

                # printout may be used for speadsheet 
                if printout:
                    print('{}, {}, {}'.format(count, i.strftime('%Y-%m-%d'), iloc_index), end='')
                    print(f', {d['Close']:.4f}, {d['Close_Prev']:.4f}, {d['Min']:.4f}, {d['Average']:.4f}, {d['Max']:.4f}', end='')
                    print(f', {d['RSI']:.4f}, {d['RSI_Prev']:.4f}, {d['RSI_Prev2']:.4f}, {d['RSI_Prev3']:.4f}, {d['RSI_Prev4']:.4f}')
        else:
            consecutive_days = 0
    
    return data

#### Prediction methods

In [7]:
extracted_data = {}

##### 1. Just a wild guess
1. Use several historical instances of RSI just raised above 70. Calculate the average of price_max (near future max price) divde today Close Price, $\overline{\frac{price\_max}{price}}$
2. Multiple an adjust_factor (likely less than 1) to increase the chance to sell rather than overpriced.
3. Multiple the step 2 value to today Close Price as a prediction of near future maximum price.

$$predict\_max\_price = (1 + adjust\_factor(\overline{\frac{price\_max}{price}} - 1)) close\_price$$


In [8]:
extracted_data['AAPL'] = extract_RSI70_data(dfs['AAPL'], printout=True)

count, date, index, close, close_prev, price_min, price_average, price_max, rsi, rsi_prev, rsi_prev2, rsi_prev3, rsi_prev4
1, 2023-01-23, 97, 139.1476, 135.9527, 123.2813, 130.0504, 143.9006, 72.1826, 68.5368, 69.8494, 59.5291, 57.2136
2, 2023-03-21, 137, 157.3047, 155.4480, 143.5079, 149.2441, 158.2627, 74.1946, 67.1708, 63.0147, 66.5820, 56.5534
3, 2023-03-29, 143, 158.7762, 155.6949, 143.5079, 151.9259, 164.1093, 71.0679, 60.2487, 63.9399, 63.2535, 65.3697
4, 2023-04-06, 149, 162.6180, 161.7291, 146.6584, 156.0733, 164.1093, 71.3906, 67.5543, 76.8707, 79.0419, 79.7770
5, 2023-05-01, 165, 167.4868, 167.5757, 158.1145, 163.0759, 171.4175, 72.6198, 68.5950, 58.5499, 50.0000, 45.3592
6, 2023-06-02, 188, 178.9531, 178.1026, 163.7340, 171.6732, 178.9630, 75.1201, 69.6300, 63.9256, 68.1963, 56.9075
7, 2023-06-12, 194, 181.7618, 178.9630, 169.6668, 174.2672, 183.9573, 72.5861, 64.9794, 64.4654, 63.7240, 69.9220
8, 2023-06-27, 204, 185.9847, 183.2255, 171.0810, 179.8167, 191.8295, 71.7661, 6

In [9]:
def predict_max_price(data, num_sample, adjust_factor=0.5):
    hit = 0
    miss = 0
    predict_div_max_array = []
    for i, d in enumerate(data):
        if i < num_sample:
            d['Predict1'] = np.nan
            continue
        sum = 0
        for j in range(num_sample):
            sum += data[i-j-1]['Max'] / data[i-j-1]['Close']
        max_div_price_average = sum / num_sample
        d['Predict1'] = d['Close'] * (1 + adjust_factor * (max_div_price_average - 1))
        if d['Predict1'] < d['Max']:
            hit += 1
            predict_div_max_array.append(d['Predict1'] / d['Max'])
        else:
            miss += 1
    return hit / (hit + miss), np.array(predict_div_max_array).mean()

In [10]:
hit_rate, predict_div_max_array_mean = predict_max_price(extracted_data['AAPL'], 5, adjust_factor=0.5)
hit_rate, predict_div_max_array_mean


(0.6206896551724138, np.float64(0.9826925643312815))

In [11]:
for s in SYMBOLS:
    extracted_data[s] = extract_RSI70_data(dfs[s], printout=False)
    hit_rate, predict_div_max_array_mean = predict_max_price(extracted_data[s], 5, adjust_factor=0.5)
    print(f'{s}, hit_rate:{hit_rate:.4f}, predict/max:{predict_div_max_array_mean:.4f}')

AAPL, hit_rate:0.6207, predict/max:0.9827
AMZN, hit_rate:0.6552, predict/max:0.9816
GOOG, hit_rate:0.6000, predict/max:0.9799
META, hit_rate:0.7059, predict/max:0.9764
MSFT, hit_rate:0.6944, predict/max:0.9853
NVDA, hit_rate:0.6000, predict/max:0.9489
TSLA, hit_rate:0.4118, predict/max:0.9143


##### 2. Neural network
1. Believe that each stock has its own characteristic. Therefore, the neural network is trained per stock.
2. For every instance of RSI $\ge$ 70, the extract data and near future max price are used as input and output to train the network.
3. Understand that the train data is insufficient. Nevertheless, give it a try.

In [12]:
models = {}

In [13]:
class SimpleNet(nn.Module):
    def __init__(self, input_size):
        super(SimpleNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()  # to keep output between 0 and 1
        )
    
    def forward(self, x):
        return self.net(x) * 2  # scale output to [0, 2]


Train

In [14]:
def train_model(model, inputs, outputs, epochs=100, loss_function=None, printout=False):
    inputs = torch.tensor(inputs, dtype=torch.float32)
    targets = torch.tensor(outputs, dtype=torch.float32)

    if loss_function is None:
        criterion = nn.MSELoss()
    else:
        criterion = loss_function
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            if printout:
                print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [15]:
class AsymmetricMSELoss(nn.Module):
    def __init__(self, alpha=2.0):  # alpha > 1 means more penalty on over-prediction
        super().__init__()
        self.alpha = alpha

    def forward(self, y_pred, y_true):
        diff = y_pred - y_true
        loss = torch.where(
            diff > 0,
            self.alpha * diff ** 2,  # over-prediction penalty
            diff ** 2                # under-prediction penalty
        )
        return loss.mean()

# Example usage:
# criterion = AsymmetricMSELoss(alpha=3.0)  # penalize over-prediction 3x more

# y_true = torch.tensor([10.0, 12.0, 15.0])
# y_pred = torch.tensor([11.0, 10.0, 17.0])

# loss = criterion(y_pred, y_true)
# print(loss.item())


Evaluate

In [16]:
def evaluate_model(model, inputs):
    model.eval()
    with torch.no_grad():
        sample_input = torch.tensor(inputs, dtype=torch.float32)  # shape (1, N)
        predictions = model(sample_input)
        # print(predictions)
    return predictions

Verify

In [17]:
def verify_predicts(tune_result, predictions, factor, format='key_value'):
    if format == 'cvs':
        print('date, pred, pred_price, max, close, max/pred_price, pred_price*factor')
    for i, p in enumerate(predictions):
        pred_price = p.item() * tune_result[i]['Close']
        if format == 'key_value':
            print('date={}, pred={:.4f}, pred_price={:.2f}, max={:.2f}, close={:.2f}, max/pred_price={:.4f}, pred_price*factor={:.2f}'.format(
                tune_result[i]['Date'], 
                p.item(), 
                pred_price,
                tune_result[i]['Max'],
                tune_result[i]['Close'],
                tune_result[i]['Max'] / pred_price,
                pred_price * factor))
        elif format == 'cvs':
            print('{}, {:.4f}, {:.2f}, {:.2f}, {:.2f}, {:.4f}, {:.2f}'.format(
                tune_result[i]['Date'],
                p.item(),
                pred_price,
                tune_result[i]['Max'],
                tune_result[i]['Close'],
                tune_result[i]['Max'] / pred_price,
                pred_price * factor))


Convert data to inputs outputs format

In [18]:
def convert_data_to_inputs_outputs_with_max_look_back(data, num_max_look_back=4, num_rsi=5):
    inputs = []
    outputs = []
    max_look_back_price = [0] * num_max_look_back
    max_look_back_index = [0] * num_max_look_back
    max_look_back_slope = [0] * num_max_look_back
    for i, d in enumerate(data):
        if i >= num_max_look_back:
            for j in range(num_max_look_back - 1):
                max_look_back_slope[j] = ((max_look_back_price[j+1] - max_look_back_price[j]) / d['Close']) / \
                    ((max_look_back_index[j+1] - max_look_back_index[j]) / TRADE_DAYS_PER_YEAR)
            max_look_back_slope[num_max_look_back - 1] = ((d['Close'] - max_look_back_price[num_max_look_back - 1]) / d['Close']) / \
                ((d['Index'] - max_look_back_index[num_max_look_back - 1]) / TRADE_DAYS_PER_YEAR)
            rsis = []
            if num_rsi >= 1:
                rsis.append(d['RSI']/100)
            if num_rsi >= 2:
                rsis.append(d['RSI_Prev']/100)
            if num_rsi >= 3:
                rsis.append(d['RSI_Prev2']/100)
            if num_rsi >= 4:
                rsis.append(d['RSI_Prev3']/100)
            if num_rsi >= 5:
                rsis.append(d['RSI_Prev4']/100)

            inputs.append([d['Min']/d['Close'], 
                        d['Average']/d['Close'],
                        d['Volume_log_Norm']] + 
                        rsis + 
                        max_look_back_slope)
            outputs.append([d['Max']/d['Close']])

        max_look_back_price.pop(0)
        max_look_back_price.append(d['Max'])
        max_look_back_index.pop(0)
        max_look_back_index.append(d['Index'])

    return inputs, outputs

In [19]:
NUM_BASIC_INPUTS = 3
NUM_MAX_LOOK_BACK = 4
NUM_RSI = 5
NUM_RESERVE_TEST = 5
PREDICT_FACTOR = 0.98

models['AAPL'] = SimpleNet(NUM_BASIC_INPUTS + NUM_MAX_LOOK_BACK + NUM_RSI)
# extracted_data['AAPL'] = extract_RSI70_data(dfs['AAPL'])  # already extracted 
inputs, outputs = convert_data_to_inputs_outputs_with_max_look_back(extracted_data['AAPL'], NUM_MAX_LOOK_BACK, NUM_RSI)
train_model(models['AAPL'], inputs[:-NUM_RESERVE_TEST], outputs[:-NUM_RESERVE_TEST], loss_function=AsymmetricMSELoss(alpha=2.0), printout=True)   # train without last NUM_RESERVE_TEST data
predictions = evaluate_model(models['AAPL'], inputs[-NUM_RESERVE_TEST:])   # evaluate last NUM_RESERVE_TEST data
verify_predicts(extracted_data['AAPL'][-NUM_RESERVE_TEST:], predictions, PREDICT_FACTOR)

Epoch 10, Loss: 0.0009
Epoch 20, Loss: 0.0005
Epoch 30, Loss: 0.0003
Epoch 40, Loss: 0.0001
Epoch 50, Loss: 0.0001
Epoch 60, Loss: 0.0001
Epoch 70, Loss: 0.0001
Epoch 80, Loss: 0.0000
Epoch 90, Loss: 0.0000
Epoch 100, Loss: 0.0000
date=2025-07-02, pred=1.0187, pred_price=216.17, max=213.31, close=212.20, max/pred_price=0.9868, pred_price*factor=211.85
date=2025-07-18, pred=1.0056, pred_price=212.12, max=214.16, close=210.94, max/pred_price=1.0096, pred_price*factor=207.87
date=2025-08-13, pred=1.0163, pred_price=237.14, max=233.33, close=233.33, max/pred_price=0.9839, pred_price*factor=232.39
date=2025-08-18, pred=1.0208, pred_price=235.69, max=233.33, close=230.89, max/pred_price=0.9900, pred_price*factor=230.98
date=2025-08-21, pred=1.0214, pred_price=229.71, max=232.78, close=224.90, max/pred_price=1.0133, pred_price*factor=225.12


In [20]:
verify_predicts(extracted_data['AAPL'][-NUM_RESERVE_TEST:], predictions, PREDICT_FACTOR, format='cvs')

date, pred, pred_price, max, close, max/pred_price, pred_price*factor
2025-07-02, 1.0187, 216.17, 213.31, 212.20, 0.9868, 211.85
2025-07-18, 1.0056, 212.12, 214.16, 210.94, 1.0096, 207.87
2025-08-13, 1.0163, 237.14, 233.33, 233.33, 0.9839, 232.39
2025-08-18, 1.0208, 235.69, 233.33, 230.89, 0.9900, 230.98
2025-08-21, 1.0214, 229.71, 232.78, 224.90, 1.0133, 225.12


Train and evaluate for all SYMBOLS. evaluate_model() will evaluate all data while only the last NUM_RESERVE_TEST are unseen in training.

Store predicted maximum price to ```extracted_data[ticker][i]['Predict2']```.

In [21]:
for s in SYMBOLS:
    models[s] = SimpleNet(NUM_BASIC_INPUTS + NUM_MAX_LOOK_BACK + NUM_RSI)
    # extracted_data[s] = extract_RSI70_data(dfs[s], printout=False)
    inputs, outputs = convert_data_to_inputs_outputs_with_max_look_back(extracted_data[s], NUM_MAX_LOOK_BACK, NUM_RSI)
    train_model(models[s], inputs[:-NUM_RESERVE_TEST], outputs[:-NUM_RESERVE_TEST], loss_function=AsymmetricMSELoss(alpha=2.0), printout=False)   # train without last NUM_RESERVE_TEST data
    predictions = evaluate_model(models[s], inputs)   # evaluate all except first few which do not have enough max_look_back
    for i in range(len(predictions)):
        extracted_data[s][NUM_MAX_LOOK_BACK+i]['Predict2'] = predictions[i].item() * extracted_data[s][NUM_MAX_LOOK_BACK+i]['Close'] * PREDICT_FACTOR
    print(s)
    verify_predicts(extracted_data[s][NUM_MAX_LOOK_BACK:], predictions, PREDICT_FACTOR, format='cvs')


AAPL
date, pred, pred_price, max, close, max/pred_price, pred_price*factor
2023-05-01, 1.0020, 167.83, 171.42, 167.49, 1.0214, 164.47
2023-06-02, 1.0176, 182.11, 178.96, 178.95, 0.9827, 178.47
2023-06-12, 1.0034, 182.38, 183.96, 181.76, 1.0086, 178.74
2023-06-27, 1.0125, 188.31, 191.83, 185.98, 1.0187, 184.55
2023-07-17, 1.0038, 192.57, 192.95, 191.85, 1.0020, 188.72
2023-07-28, 1.0068, 194.98, 194.28, 193.67, 0.9964, 191.08
2023-09-05, 1.0101, 189.75, 187.86, 187.86, 0.9900, 185.96
2023-11-10, 1.0087, 186.45, 188.12, 184.84, 1.0090, 182.72
2023-11-14, 1.0203, 189.65, 189.84, 185.87, 1.0010, 185.86
2023-11-30, 1.0039, 189.08, 192.64, 188.36, 1.0188, 185.30
2024-01-25, 1.0046, 193.43, 193.54, 192.54, 1.0006, 189.56
2024-05-07, 1.0071, 182.38, 186.35, 181.10, 1.0217, 178.74
2024-05-31, 1.0083, 192.73, 195.75, 191.14, 1.0157, 188.87
2024-06-11, 1.0437, 214.96, 215.42, 205.95, 1.0021, 210.66
2024-06-27, 1.0286, 218.95, 225.03, 212.86, 1.0278, 214.57
2024-07-01, 1.0593, 228.28, 227.36, 215.

##### 3. Baseline
When RSI $\ge$ 70, use the Close Price as the selling price for the next few trading dates.

#### simulate investment and calculate the return
Assume gain from recent average price to selling price if predicted price is lower than near future maximum price.

In [22]:
def simulate_investment(data, amount=1000):
    amount_wild = amount
    amount_model = amount
    amount_baseline = amount
    amount_ideal = amount
    for i, d in enumerate(data):
        if d['Predict1'] <= d['Max']:
            amount_wild = d['Predict1'] / d['Average'] * amount_wild
        if d['Predict2'] <= d['Max']:
            amount_model = d['Predict2'] / d['Average'] * amount_model
        if d['Close'] <= d['Max']:
            amount_baseline = d['Close'] / d['Average'] * amount_baseline
        amount_ideal = d['Max'] / d['Average'] * amount_ideal
    return amount_wild, amount_model, amount_baseline, amount_ideal

In [23]:
for s in SYMBOLS:
    amount = {}
    amount['wild'], amount['model'], amount['baseline'], amount_ideal = simulate_investment(extracted_data[s][-NUM_RESERVE_TEST:])
    sorted_methods = sorted(amount, key=lambda x: amount[x], reverse=True)
    print(f'{s}: wild={amount['wild']:.2f}, model={amount['model']:.2f}, baseline={amount['baseline']:.2f}, ideal={amount_ideal:.2f}', end='')
    print(f', {sorted_methods[0]} > {sorted_methods[1]} > {sorted_methods[2]}')

AAPL: wild=1193.48, model=1203.80, baseline=1284.49, ideal=1371.16, baseline > model > wild
AMZN: wild=1019.42, model=1120.00, baseline=1198.14, ideal=1230.43, baseline > model > wild
GOOG: wild=1197.68, model=1177.86, baseline=1238.37, ideal=1414.37, baseline > wild > model
META: wild=1398.81, model=1374.65, baseline=1407.76, ideal=1871.31, baseline > wild > model
MSFT: wild=1094.50, model=1182.49, baseline=1166.64, ideal=1285.04, model > baseline > wild
NVDA: wild=1203.79, model=1341.68, baseline=1391.04, ideal=1735.07, baseline > model > wild
TSLA: wild=1167.26, model=1429.36, baseline=1663.70, ideal=2064.93, baseline > model > wild


Observation
1. This simulation favors approach which can make more transactions. Therefore, baseline is the best in most cases while wild guess method and neural network draw.
2. For Amazon, baseline performance is quite close to the ideal.

In [24]:
pd.__version__

'2.3.2'

In [25]:
yf.__version__

'0.2.65'

In [26]:
import plotly
plotly.__version__

'6.3.0'

In [27]:
import nbformat
nbformat.__version__

'5.10.4'

In [28]:
import torch
torch.__version__

'2.8.0'